# Create Channel dimension from sources
## Take Bronze data and load it into Silver

In [0]:
%python
#Set the operation needed when processing data. 
    #Daily will load only changes
    #Reload will flush the data and reload
    # Rebuild will drop and recreate the tables
    
dbutils.widgets.removeAll()
#Operations
dbutils.widgets.dropdown('Operations', 'Daily',['Daily','Reload','Rebuild'])

operations = dbutils.widgets.get('Operations')


## Rebuild the table if needed

In [0]:
%python
if operations =='Rebuild':
    rebsql = f'''
    CREATE OR REPLACE TABLE neighbor.silver.channel (
        ChannelID INTEGER,
        ChannelNAme STRING,
        UPDATEDATEUTC TIMESTAMP
      )
    '''
    spark.sql(rebsql)
else:
    print("This operation is skipped")

## Reload data if needed

In [0]:
%python
if operations == 'Reload':
    asppsql = f'''
TRUNCATE TABLE neighbor.silver.channel
'''
    spark.sql(asppsql)
else: 
    print("This cell is skipped.")


## Load data into silver schema

In [0]:
merSQL = '''
MERGE INTO neighbor.silver.channel AS tgt
    USING (
        select distinct 
    row_number() over (order by Channel) as channelid,
    ch.channel as ChannelNAme
    from
    ( SELECT DISTINCT first_attribution_channel as Channel
    from neighbor.bronze.all_search_events
    )ch
) AS stg
ON tgt.ChannelID = stg.ChannelID
WHEN MATCHED THEN UPDATE SET
    tgt.ChannelNAme = stg.ChannelNAme,
    tgt.UPDATEDATEUTC = to_utc_timestamp(CURRENT_TIMESTAMP, 'UTC')
WHEN NOT MATCHED THEN INSERT (
    ChannelID, ChannelNAme,UPDATEDATEUTC
) 
VALUES (
    stg.ChannelID,
    stg.ChannelNAme,
    to_utc_timestamp(CURRENT_TIMESTAMP, 'UTC')
)
'''
spark.sql(merSQL)

In [0]:
%sql
SELECT * FROM neighbor.silver.channel